# Savvy Zealy Airdrop: Round 1

In [1]:
import json
import pandas as pd

I scraped a snapshot of Zealy using their API on November 13, 2023.

Load the snapshot and drop columns that are not necessary for the airdrop.

In [2]:
with open('data/p0.json', 'r') as f:
    df0 = pd.DataFrame.from_dict(json.load(f)['leaderboard'], orient='columns')

with open('data/p1.json', 'r') as f:
    df1 = pd.DataFrame.from_dict(json.load(f)['leaderboard'], orient='columns')

df = pd.concat([df0, df1])
df = df.drop(['userId', 'name', 'avatar', 'addresses', 'twitterId', 'discord', 'twitter', 'discordId', 'connectedWallet'], axis=1)

Next, we identify accounts for the airdrop and tally the XP total.

In [3]:
df = df[df['xp'] > 2000]
round_svy_total = 20000
total_xp = df['xp'].sum()
total_xp

2123935

Distribute this round's 20k SVY across eligible accounts and double-check that it still sums to 20k SVY.

In [4]:
df['proportion'] = df['xp'] / total_xp
df['amount'] = df['proportion'] * round_svy_total
df['amount'].sum()

20000.0

Write the results of round1.

In [5]:
df.to_csv('data/round1.csv')

Done